# Getting Started

## Starting sc3nb

In [1]:
import sc3nb as scn

To startup sc3nb (sclang, scsynth, and a python OSC server) use `startup` which will return a `SC` instance which is the central SuperCollider Interface

In [2]:
sc = scn.startup()  # see Configuration of sc3nb startup below for more details

<IPython.core.display.Javascript object>

Starting sclang process... 

Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... 

Done.


You can produce a test sound with `blip`, which should relax any tensions whether the server is up and running. This sound should be played by the default server start.

In [3]:
sc.server.blip()

`sc` provides you the interfaces for

* `scsynth` via `sc.server`
* `sclang` via `sc.lang`

**Configuration of sc3nb startup**:

- Specifying the excutable location
  - We try to find the sclang and scsynth executables in the `$PATH` environment variable and also in the default installation directories
      - On macOS they reside in `/Applications/SuperCollider.app/Contents` in the folders MacOS and Resources. <br>
    To add these paths to your `$PATH`, simply add to your ~/.profile, e.g. (please adapt to your installation): <br>
    `PATH=$PATH:/Applications/SuperCollider.app/Contents/MacOS:/Applications/SuperCollider.app/Contents/Resources`  
      - On Windows they reside in your Installation folder f.e `C:\Program Files\SuperCollider-3.x.x`

  - If the executables are not found, you can specify them with `sclang_path` / `scsynth_path` e.g. `sclang_path="/path/to/sclang-containing-dir/sclang"`
- You could also only use **scsynth** only and don't start **sclang** with `start_sclang=False`
- See `help(scn.startup)` for all options

In [4]:
# help(scn.startup)

## Basic examples

### Use Python implementations to control the SuperCollider audio server

Create and control SuperCollider Synths

In [5]:
syn = scn.Synth()
syn

<Synth(20001) 'default' s {}>

In [6]:
syn.freq = 800
syn

<Synth(20001) 'default' s {'freq': 800}>

In [7]:
syn.free()  # or use the ctrl-. (cmd-.) shortcut in Jupyter notebooks
syn

<Synth(20001) 'default' f {'freq': 800}>

Send SynthDefs with python code injection

In [8]:
synth_dur = 0.5  # you can use python variables in SynthDefs by prepending ^
synth_def = scn.SynthDef('random', """{ |out|
    var line;
    line = Line.kr(1, 0, ^synth_dur, doneAction: Done.freeSelf);
    Out.ar(out, SinOsc.ar(Rand(400, 800), 0, 0.2) * line);
}""")
synth_def.add()
synth_def  # Note that the representation has synth_dur already injected

SynthDef('random', { |out|
    var line;
    line = Line.kr(1, 0, 0.5, doneAction: Done.freeSelf);
    Out.ar(out, SinOsc.ar(Rand(400, 800), 0, 0.2) * line);
})

In [9]:
scn.Synth("random")

<Synth(20002) 'random' s {}>

Load a file as Buffer and play it

In [10]:
buf = scn.Buffer().read("./media/blip.wav")
buf

<Buffer(128) on ('127.0.0.1', 57110): 2 x 43008 @ 44100 Hz = 0.975s allocated using mode 'file'>

In [11]:
buf.play()

<Synth(20003) 'sc3nb_playbuf_128' s {}>

In [12]:
buffer_data = buf.to_array()
print(buffer_data.shape)
buffer_data

(43008, 2)


array([[-0.00189209, -0.00189209],
       [-0.00430298, -0.00430298],
       [-0.00708008, -0.00708008],
       ...,
       [ 0.19314575,  0.19314575],
       [ 0.18963623,  0.18963623],
       [ 0.18585205,  0.18585205]])

More examples about the above used SuperCollider objects can be found in the respective User Guide sections 
* [Nodes (Synths and Groups)](./supercollider-objects/node-examples.ipynb)
* [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
* [Buffer](./supercollider-objects/buffer-examples.ipynb)

### Use OSC to control the SuperCollider audio server

Send OSC Messages with [SuperCollider Commands](https://doc.sccode.org/Reference/Server-Command-Reference.html) and receive replies

In [13]:
sc.server.msg("/status")  # waits for reply message and returns it

(1,
 9,
 1,
 9,
 117,
 0.27066338062286377,
 0.47807106375694275,
 44100.0,
 43548.905905256324)

We offer also high level versions for the commands. 

In [14]:
sc.server.status()

ServerStatus(num_ugens=9, num_synths=1, num_groups=9, num_synthdefs=117, avg_cpu=0.26837432384490967, peak_cpu=0.47807106375694275, nominal_sr=44100.0, actual_sr=43563.0405712431)

Create OSC Bundles using SuperCollider Commands directly

In [15]:
with sc.server.bundler() as bundler:
    bundler.add(0.2, "/s_new", ["s1", -1, 0, 0])
    bundler.add(0.5, "/s_new", ["s1", -1, 0, 0, "freq", 800, "dur", 0.1])
bundler.messages()

{0.2: [<OSCMessage("/s_new", ['s1', -1, 0, 0])>],
 0.5: [<OSCMessage("/s_new", ['s1', -1, 0, 0, 'freq', 800, 'dur', 0.10000000149011612])>]}

or create OSC Bundles with the high level Python implementations

In [16]:
with sc.server.bundler() as bundler:
    bundler.wait(0.2)
    scn.Synth("s1")
    bundler.wait(0.3)
    scn.Synth("s1", {"freq": 800, "dur": 0.1})
bundler.messages()

{0.2: [<OSCMessage("/s_new", ['s1', 20004, 0, 67108865])>],
 0.5: [<OSCMessage("/s_new", ['s1', 20005, 0, 67108865, 'freq', 800, 'dur', 0.10000000149011612])>]}

[More OSC communication examples](./osc-communication-examples.ipynb)

### Use the SuperCollider Language from Python

Execute SuperCollider Language Code

In [17]:
breakfast = "eggs"
sc.lang.cmd('^breakfast.scramble;')

-> ggse


or via the **%sc** IPython magic

In [18]:
%sc ^breakfast.scramble

-> sgeg


Get results from SuperCollider Language in python with `cmdg` or **%scg**

In [19]:
x = 5
value = %scg (1..^x)
print(f"received {value} with type {type(value)}")

-> [ 1, 2, 3, 4, 5 ]
received [1, 2, 3, 4, 5] with type <class 'list'>


[More sclang in sc3nb examples](./osc-communication-examples.ipynb)

### Stoping the synthesis

There are multiple options to stop the playing audio synthesis

In [20]:
%sc x = Synth.new("default")  // starting a Synth with sclang

-> Synth('default' : 1000)


* to stop all playing synths either use CMD-. (in Jupyter Command mode).

* It is a shortcut for the ´free_all´ method of the default server

In [21]:
sc.server.free_all()

* or you could also use sclang

In [22]:
%sc s.freeAll

-> sc3nb_remote


### Combined usage

It is also possible to mix the different interaction styles.

* create a Synth with sclang and get the nodeid of the Synth via the **%scg** IPython magic

In [23]:
nodeid = %scg x = Synth("default"); x.nodeID;

-> 1001


* create the corresponding Python Synth instance

In [24]:
synth = scn.Synth("default", nodeid=nodeid, new=False)
synth

<Synth(1001) 'default'  {}>

* Inspect and control the Synth from Python

In [25]:
synth.synth_desc

{'out': SynthArgument(name='out', rate='scalar', default=0.0),
 'freq': SynthArgument(name='freq', rate='control', default=440.0),
 'amp': SynthArgument(name='amp', rate='control', default=0.10000000149011612),
 'pan': SynthArgument(name='pan', rate='control', default=0.0),
 'gate': SynthArgument(name='gate', rate='control', default=1.0)}

In [26]:
synth.freq

440.0

In [27]:
synth.freq *= 2
synth.freq

880.0

In [28]:
synth

<Synth(1001) 'default' ~ {'freq': 880.0}>

* send a OSC Message to free the Synth

In [29]:
sc.server.msg("/n_free", nodeid)

In [30]:
synth

<Synth(1001) 'default' ~ {'freq': 880.0}>

### Exiting sc3nb

To shut down the server and sclang subprocesses

In [31]:
sc.exit()  

## Further information and advanced examples

For more details on the specific parts please refer to the corresponding sections of the User Guide.

* Usage of [SuperCollider Language (sclang) in sc3nb](./sclang-examples.ipynb)


* Usage of the SuperCollider Objects python interface
  * [Server](./supercollider-objects/server-examples.ipynb)
  * [Nodes (Synth and Group)](./supercollider-objects/node-examples.ipynb)
  * [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
  * [Buffer](./supercollider-objects/buffer-examples.ipynb)
  * [Bus](./supercollider-objects/bus-examples.ipynb)
  * [Recorder](./supercollider-objects/recorder-examples.ipynb)


* Usage of [common SuperCollider helper functions](./helper-examples.ipynb)
* Usage of [OSC in sc3nb](./osc-communication-examples.ipynb)
